In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pickle
import requests

In [3]:
raws_data = pd.read_csv('recent_wildfires.csv')

In [4]:
df = raws_data.filter(['X', 'Y', 'FireCause', 'FireDiscoveryDateTime'])

In [6]:
# Create a dataset of non human fires
non_human_fires = df[df['FireCause']!='Human']
non_human_fires = pd.DataFrame(non_human_fires)

In [7]:
non_human_fires[['temp', 'humidity', 'precip', 'windspeed', 'solarradiation']] = ''

In [8]:
for index, row in non_human_fires.iterrows():
    fire_date = row["FireDiscoveryDateTime"].split(" ")[0].replace("/","-")
    lng, lat = str(row["Y"]), str(row["X"])
    
    url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/" \
        + lng + "," + lat + "/" + fire_date + "/" + fire_date \
        + "?unitGroup=us&include=days&key=N34H8P28WBZH5ADUJAVME467A&contentType=json"
    response = requests.request("GET", url)
    if response.status_code!=200:
        print(index, 'Unexpected Status code: ', response.status_code, response.text)
        break
    else:
        # print(response.text)
        wd = response.json()
        wd = wd['days'][0]
        non_human_fires.at[index, 'temp'] = wd['temp'] if 'temp' in wd else np.nan
        non_human_fires.at[index, 'humidity'] = wd['humidity'] if 'humidity' in wd else np.nan
        non_human_fires.at[index, 'precip'] = wd['precip'] if 'precip' in wd else np.nan
        non_human_fires.at[index, 'windspeed'] = wd['windspeed'] if 'windspeed' in wd else np.nan
        non_human_fires.at[index, 'solarradiation'] = wd['solarradiation'] if 'solarradiation' in wd else np.nan
        

In [9]:
non_human_fires = non_human_fires.drop('FireCause', axis=1)

In [10]:
non_human_fires['Date'] = pd.to_datetime(non_human_fires['FireDiscoveryDateTime']).astype(int) / 10**12
non_human_fires = non_human_fires.drop('FireDiscoveryDateTime', axis=1)

In [11]:
non_human_fires = non_human_fires[['Y', 'X', 'temp', 'Date', 'precip', 'humidity', 'solarradiation', 'windspeed']]
loaded_model = None
with open('wildfire_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

In [13]:
import csv

with open('/home/revanth/tamids/output.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    # writer.writerow(['Y', 'X', 'prediction'])
    for index, row in non_human_fires.iterrows():
        incident = row.to_frame().T
        prediction = loaded_model.predict(incident)
        prediction_prob = loaded_model.predict_proba(incident)
        print("data prediction of index:", index, "is: ", prediction, prediction_prob)
        
        writer.writerow([row['Y'],row['X'], int(prediction_prob[0][1] * 10)])

data prediction of index: 0 is:  [ True] [[0. 1.]]
data prediction of index: 1 is:  [ True] [[0.01 0.99]]
data prediction of index: 2 is:  [ True] [[0.19 0.81]]
data prediction of index: 3 is:  [ True] [[0.41 0.59]]
data prediction of index: 5 is:  [ True] [[0.13 0.87]]
data prediction of index: 10 is:  [ True] [[0. 1.]]
data prediction of index: 11 is:  [False] [[0.57 0.43]]
data prediction of index: 12 is:  [False] [[0.57 0.43]]
data prediction of index: 13 is:  [ True] [[0.33 0.67]]
data prediction of index: 14 is:  [False] [[0.61 0.39]]
data prediction of index: 16 is:  [False] [[0.61 0.39]]
data prediction of index: 17 is:  [ True] [[0. 1.]]
data prediction of index: 18 is:  [ True] [[0.4 0.6]]
data prediction of index: 19 is:  [ True] [[0.46 0.54]]
data prediction of index: 20 is:  [ True] [[0. 1.]]
data prediction of index: 21 is:  [False] [[0.75 0.25]]
data prediction of index: 22 is:  [ True] [[0.01 0.99]]
data prediction of index: 23 is:  [False] [[0.65 0.35]]
data prediction